In [99]:
import keras.models
import pandas as pd
import numpy as np
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications import mobilenet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import obtain_input_shape
from keras.models import Model
from keras.optimizers import adam_v2
from PIL import Image

from keras.layers import Dense, Activation, Flatten

In [100]:
from keras.applications.mobilenet import MobileNet

base_model = MobileNet(weights='imagenet', include_top=False)

In [101]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
preds = Dense(3, activation='softmax')(x)


In [102]:
model = Model(inputs=base_model.input, outputs=preds)

In [103]:
for i, layer in enumerate(model.layers): print(i, layer.name)

0 input_7
1 conv1
2 conv1_bn
3 conv1_relu
4 conv_dw_1
5 conv_dw_1_bn
6 conv_dw_1_relu
7 conv_pw_1
8 conv_pw_1_bn
9 conv_pw_1_relu
10 conv_pad_2
11 conv_dw_2
12 conv_dw_2_bn
13 conv_dw_2_relu
14 conv_pw_2
15 conv_pw_2_bn
16 conv_pw_2_relu
17 conv_dw_3
18 conv_dw_3_bn
19 conv_dw_3_relu
20 conv_pw_3
21 conv_pw_3_bn
22 conv_pw_3_relu
23 conv_pad_4
24 conv_dw_4
25 conv_dw_4_bn
26 conv_dw_4_relu
27 conv_pw_4
28 conv_pw_4_bn
29 conv_pw_4_relu
30 conv_dw_5
31 conv_dw_5_bn
32 conv_dw_5_relu
33 conv_pw_5
34 conv_pw_5_bn
35 conv_pw_5_relu
36 conv_pad_6
37 conv_dw_6
38 conv_dw_6_bn
39 conv_dw_6_relu
40 conv_pw_6
41 conv_pw_6_bn
42 conv_pw_6_relu
43 conv_dw_7
44 conv_dw_7_bn
45 conv_dw_7_relu
46 conv_pw_7
47 conv_pw_7_bn
48 conv_pw_7_relu
49 conv_dw_8
50 conv_dw_8_bn
51 conv_dw_8_relu
52 conv_pw_8
53 conv_pw_8_bn
54 conv_pw_8_relu
55 conv_dw_9
56 conv_dw_9_bn
57 conv_dw_9_relu
58 conv_pw_9
59 conv_pw_9_bn
60 conv_pw_9_relu
61 conv_dw_10
62 conv_dw_10_bn
63 conv_dw_10_relu
64 conv_pw_10
65 conv_pw_1

In [104]:
for layer in model.layers[:86]:
    layer.trainable = False
for layer in model.layers[86:]:
    layer.trainable = True

In [105]:
# Fitting CNN to images

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(rescale=1./255)

In [106]:
# target size is the mobilenet target size that is required (224,224)

training_set = train_datagen.flow_from_directory('../data/fruits_cnn_data/Training',
                                                 target_size=(224, 224),
                                                 batch_size=8,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('../data/fruits_cnn_data/Validation',
                                                 target_size=(224, 224),
                                                 batch_size=8,
                                                 class_mode='categorical')


# categorical cz 3 classes, if only 2 classes - binary o/p required then
# classmode = 'binary'

Found 24 images belonging to 3 classes.
Found 24 images belonging to 3 classes.


In [107]:
model_name = 'fruitdetect_model'
import datetime
print("Start time:", datetime.datetime.now())

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)
checkpoint_callback = ModelCheckpoint(model_name+ '.h5', monitor='val_loss', verbose=1, save_best_only=True)

Start time: 2022-01-23 22:56:03.123853


In [108]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [109]:
model.fit_generator(training_set,
                    steps_per_epoch=2,
                    epochs=20,
                    validation_data=test_set,
                    validation_steps=2,callbacks=[early_stopping_callback,checkpoint_callback])

print("End time:",datetime.datetime.now())

Epoch 1/20


C:\Users\sylvi\AppData\Local\Temp\ipykernel_3332\582598675.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(training_set,


2/2 [==============================] - ETA: 0s - loss: 0.5025 - accuracy: 0.5000    
Epoch 00001: val_loss improved from inf to 0.05635, saving model to fruitdetect_model.h5
2/2 [==============================] - 3s 1s/step - loss: 0.5025 - accuracy: 0.5000 - val_loss: 0.0563 - val_accuracy: 1.0000
Epoch 2/20
2/2 [==============================] - ETA: 0s - loss: 0.0284 - accuracy: 1.0000
Epoch 00002: val_loss improved from 0.05635 to 0.00981, saving model to fruitdetect_model.h5
2/2 [==============================] - 1s 939ms/step - loss: 0.0284 - accuracy: 1.0000 - val_loss: 0.0098 - val_accuracy: 1.0000
Epoch 3/20
2/2 [==============================] - ETA: 0s - loss: 0.0032 - accuracy: 1.0000
Epoch 00003: val_loss improved from 0.00981 to 0.00331, saving model to fruitdetect_model.h5
2/2 [==============================] - 1s 771ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 4/20
2/2 [==============================] - ETA: 0s - loss: 0.0015

In [110]:
# Test Image
import cv2
import numpy as np

img = cv2.imread("../data/fruits_cnn_data/Validation/Banana/6_100.jpg")

print(img)

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]


In [111]:
print("image shape original :", img.shape)

img = cv2.resize(img,(224,224))

print("image shape after resize :", img.shape)

print(img)
# img is a big matrix

image shape original : (100, 100, 3)
image shape after resize : (224, 224, 3)
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]


In [112]:
# Converting entire image frame into float
img = img.astype('float32')
img = img/255  # diving will convert image to range 0 to 1 making
            # computations easier & faster.
img = np.expand_dims(img, axis=0)
img.shape
print(img)


[[[[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  ...

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]

  [[1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]
   ...
   [1. 1. 1.]
   [1. 1. 1.]
   [1. 1. 1.]]]]


In [113]:
# Load keras model & predict

model = keras.models.load_model("./fruitdetect_model.h5")
pred = model.predict(img)
print("prediction probabilities for each class", pred)

class_detected_max_probs = np.argmax(pred)

class_dict = training_set.class_indices
print("class index dictionary ",class_dict)

key_list = list(class_dict.keys())
print("key list ", key_list)

# pass max probability index to get class
print("predicted class is :", key_list[class_detected_max_probs])


prediction probabilities for each class [[1.7939323e-09 9.9997246e-01 2.7520464e-05]]
class index dictionary  {'Apple Braeburn': 0, 'Banana': 1, 'Dates': 2}
key list  ['Apple Braeburn', 'Banana', 'Dates']
predicted class is : Banana


In [114]:
# Another Prediction

# Import
import cv2
import numpy as np

# REad test image
img = cv2.imread("../data/fruits_cnn_data/Validation/Dates/14_100.jpg")
print(img)

# Check shape & resize
print("image shape original :", img.shape)
img = cv2.resize(img,(224,224))
print("image shape after resize :", img.shape)
print(img)

# Converting entire image frame into float
img = img.astype('float32')
img = img/255  # diving will convert image to range 0 to 1 making
            # computations easier & faster.
img = np.expand_dims(img, axis=0)
img.shape
print(img)

# Predict
pred = model.predict(img)
print("prediction probabilities for each class", pred)

class_detected_max_probs = np.argmax(pred)

class_dict = training_set.class_indices
print("class index dictionary ",class_dict)

key_list = list(class_dict.keys())
print("key list ", key_list)

# pass max probability index to get class
print("predicted class is :", key_list[class_detected_max_probs])

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
image shape original : (100, 100, 3)
image shape after resize : (224, 224, 3)
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255